Comandos do PDF

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 56.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=252f98fcbf93a1dc952b9750955b6a9cdde90c1d571124afcfc10455a96226eb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('seuapp').getOrCreate()

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00344/Activity%20recognition%20exp.zip

--2022-08-05 03:31:21--  https://archive.ics.uci.edu/ml/machine-learning-databases/00344/Activity%20recognition%20exp.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 777127275 (741M) [application/x-httpd-php]
Saving to: ‘Activity recognition exp.zip’

Activity recognitio 100%[===================>] 741.13M  37.5MB/s    in 21s     

2022-08-05 03:31:42 (36.1 MB/s) - ‘Activity recognition exp.zip’ saved [777127275/777127275]



In [ ]:
!unzip /content/Activity\ recognition\ exp.zip 

Archive:  /content/Activity recognition exp.zip
   creating: Activity recognition exp/
  inflating: Activity recognition exp/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/Activity recognition exp/
  inflating: __MACOSX/Activity recognition exp/._.DS_Store  
  inflating: Activity recognition exp/Phones_accelerometer.csv  
  inflating: Activity recognition exp/Phones_gyroscope.csv  
  inflating: Activity recognition exp/readme.txt  
  inflating: __MACOSX/Activity recognition exp/._readme.txt  
  inflating: Activity recognition exp/Watch_accelerometer.csv  
  inflating: Activity recognition exp/Watch_gyroscope.csv  


In [89]:
phone_acc = spark.read.csv('Activity recognition exp/Phones_accelerometer.csv' ,header = True, inferSchema = False)
phone_gyr = spark.read.csv('Activity recognition exp/Phones_gyroscope.csv' ,header = True, inferSchema = False)
watch_acc = spark.read.csv('Activity recognition exp/Watch_accelerometer.csv' ,header = True, inferSchema = False)
watch_gyr = spark.read.csv('Activity recognition exp/Watch_gyroscope.csv' ,header = True, inferSchema = False)

Volumentria Dos Dados

In [90]:
print('Phones_accelerometer: ',phone_acc.count())
print('Phones_gyroscope: ',phone_gyr.count())
print('Watch_accelerometer: ',watch_acc.count())
print('Watch_gyroscope: ',watch_gyr.count())

Phones_accelerometer:  13062475
Phones_gyroscope:  13932632
Watch_accelerometer:  3540962
Watch_gyroscope:  3205431


Ajustar os Formatos dos Dados

In [91]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DateType, TimestampType, DoubleType, FloatType, LongType
phone_acc = phone_acc.withColumn('Index',phone_acc.Index.cast(IntegerType()))
phone_acc = phone_acc.withColumn('x',phone_acc.x.cast(FloatType()))
phone_acc = phone_acc.withColumn('y',phone_acc.y.cast(FloatType()))
phone_acc = phone_acc.withColumn('z',phone_acc.z.cast(FloatType()))
phone_acc = phone_acc.withColumn('Arrival_Time',phone_acc.Arrival_Time.cast(LongType()))
phone_acc = phone_acc.withColumn('Creation_Time',phone_acc.Arrival_Time.cast(LongType()))

phone_gyr = phone_gyr.withColumn('Index',phone_gyr.Index.cast(IntegerType()))
phone_gyr = phone_gyr.withColumn('x',phone_gyr.x.cast(FloatType()))
phone_gyr = phone_gyr.withColumn('y',phone_gyr.y.cast(FloatType()))
phone_gyr = phone_gyr.withColumn('z',phone_gyr.z.cast(FloatType()))
phone_gyr = phone_gyr.withColumn('Arrival_Time',phone_gyr.Arrival_Time.cast(LongType()))
phone_gyr = phone_gyr.withColumn('Creation_Time',phone_gyr.Arrival_Time.cast(LongType()))

watch_acc = watch_acc.withColumn('Index',watch_acc.Index.cast(IntegerType()))
watch_acc = watch_acc.withColumn('x',watch_acc.x.cast(FloatType()))
watch_acc = watch_acc.withColumn('y',watch_acc.y.cast(FloatType()))
watch_acc = watch_acc.withColumn('z',watch_acc.z.cast(FloatType()))
watch_acc = watch_acc.withColumn('Arrival_Time',watch_acc.Arrival_Time.cast(LongType()))
watch_acc = watch_acc.withColumn('Creation_Time',watch_acc.Arrival_Time.cast(LongType()))

watch_gyr = watch_gyr.withColumn('Index',watch_gyr.Index.cast(IntegerType()))
watch_gyr = watch_gyr.withColumn('x',watch_gyr.x.cast(FloatType()))
watch_gyr = watch_gyr.withColumn('y',watch_gyr.y.cast(FloatType()))
watch_gyr = watch_gyr.withColumn('z',watch_gyr.z.cast(FloatType()))
watch_gyr = watch_gyr.withColumn('Arrival_Time',watch_gyr.Arrival_Time.cast(LongType()))
watch_gyr = watch_gyr.withColumn('Creation_Time',watch_gyr.Arrival_Time.cast(LongType()))

Analise Inicial
- Preenchimento das Variaveis;
- Duplicidade dos Dados;
- Formato dos Dados;
- Identificação.

1. Preenchimento das Variaveis por Tabela

In [92]:
print("Table: Phones_accelerometer - Preenchimento")
phone_acc_na = phone_acc.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in phone_acc.columns])
phone_acc_na.show()

Table: Phones_accelerometer - Preenchimento
+-----+------------+-------------+---+---+---+----+-----+------+---+
|Index|Arrival_Time|Creation_Time|  x|  y|  z|User|Model|Device| gt|
+-----+------------+-------------+---+---+---+----+-----+------+---+
|  0.0|         0.0|          0.0|0.0|0.0|0.0| 0.0|  0.0|   0.0|0.0|
+-----+------------+-------------+---+---+---+----+-----+------+---+



In [93]:
print("Table: Phones_gyroscope - Preenchimento")
phone_gyr_na = phone_gyr.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in phone_gyr.columns])
phone_gyr_na.show()

Table: Phones_gyroscope - Preenchimento
+-----+------------+-------------+---+---+---+----+-----+------+---+
|Index|Arrival_Time|Creation_Time|  x|  y|  z|User|Model|Device| gt|
+-----+------------+-------------+---+---+---+----+-----+------+---+
|  0.0|         0.0|          0.0|0.0|0.0|0.0| 0.0|  0.0|   0.0|0.0|
+-----+------------+-------------+---+---+---+----+-----+------+---+



In [94]:
print("Table: Watch_accelerometer - Preenchimento")
watch_acc_na = watch_acc.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in watch_acc.columns])
watch_acc_na.show()

Table: Watch_accelerometer - Preenchimento
+-----+------------+-------------+---+---+---+----+-----+------+---+
|Index|Arrival_Time|Creation_Time|  x|  y|  z|User|Model|Device| gt|
+-----+------------+-------------+---+---+---+----+-----+------+---+
|  0.0|         0.0|          0.0|0.0|0.0|0.0| 0.0|  0.0|   0.0|0.0|
+-----+------------+-------------+---+---+---+----+-----+------+---+



In [95]:
print("Table: Watch_gyroscope - Preenchimento")
watch_gyr_na = watch_gyr.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in watch_gyr.columns])
watch_gyr_na.show()

Table: Watch_gyroscope - Preenchimento
+-----+------------+-------------+---+---+---+----+-----+------+---+
|Index|Arrival_Time|Creation_Time|  x|  y|  z|User|Model|Device| gt|
+-----+------------+-------------+---+---+---+----+-----+------+---+
|  0.0|         0.0|          0.0|0.0|0.0|0.0| 0.0|  0.0|   0.0|0.0|
+-----+------------+-------------+---+---+---+----+-----+------+---+



2. Duplicidade dos Dados

In [96]:
print("Table: Phones_accelerometer - Duplicity")
if phone_acc.count() > phone_acc.dropDuplicates().count():
    print('Data has duplicates')
  
for cols in ['Index','Arrival_Time','Creation_Time']:
  if phone_acc.count() > phone_acc.dropDuplicates([cols,'User','Model','Device']).count():
      print('Data has duplicates: ', cols)
      #raise ValueError('Data has duplicates')

Table: Phones_accelerometer - Duplicity
Data has duplicates:  Arrival_Time
Data has duplicates:  Creation_Time


In [97]:
print("Table: Phones_gyroscope - Duplicity")
if phone_gyr.count() > phone_gyr.dropDuplicates().count():
    print('Data has duplicates')
  
for cols in ['Index','Arrival_Time','Creation_Time']:
  if phone_gyr.count() > phone_gyr.dropDuplicates([cols,'User','Model','Device']).count():
      print('Data has duplicates: ', cols)
      #raise ValueError('Data has duplicates')

Table: Phones_gyroscope - Duplicity
Data has duplicates:  Arrival_Time
Data has duplicates:  Creation_Time


In [98]:
print("Table: Watch_accelerometer - Duplicity")
if watch_acc.count() > watch_acc.dropDuplicates().count():
    print('Data has duplicates')
  
for cols in ['Index','Arrival_Time','Creation_Time']:
  if watch_acc.count() > watch_acc.dropDuplicates([cols,'User','Model','Device']).count():
      print('Data has duplicates: ', cols)
      #raise ValueError('Data has duplicates')

Table: Watch_accelerometer - Duplicity
Data has duplicates:  Arrival_Time
Data has duplicates:  Creation_Time


In [99]:
print("Table: Watch_gyroscope - Duplicity")
if watch_gyr.count() > watch_gyr.dropDuplicates().count():
    print('Data has duplicates')
  
for cols in ['Index','Arrival_Time','Creation_Time']:
  if watch_gyr.count() > watch_gyr.dropDuplicates([cols,'User','Model','Device']).count():
      print('Data has duplicates: ', cols)
      #raise ValueError('Data has duplicates')

Table: Watch_gyroscope - Duplicity
Data has duplicates:  Arrival_Time
Data has duplicates:  Creation_Time


3. Formato dos Dados

In [100]:
print("Table: Phones_accelerometer - Format")
phone_acc.printSchema()

Table: Phones_accelerometer - Format
root
 |-- Index: integer (nullable = true)
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- User: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Device: string (nullable = true)
 |-- gt: string (nullable = true)



In [101]:
print("Table: Phones_gyroscope - Format")
phone_gyr.printSchema()

Table: Phones_gyroscope - Format
root
 |-- Index: integer (nullable = true)
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- User: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Device: string (nullable = true)
 |-- gt: string (nullable = true)



In [102]:
print("Table: Watch_accelerometer - Format")
watch_acc.printSchema()

Table: Watch_accelerometer - Format
root
 |-- Index: integer (nullable = true)
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- User: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Device: string (nullable = true)
 |-- gt: string (nullable = true)



In [103]:
print("Table: Watch_gyroscope - Format")
watch_gyr.printSchema()

Table: Watch_gyroscope - Format
root
 |-- Index: integer (nullable = true)
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- User: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Device: string (nullable = true)
 |-- gt: string (nullable = true)



4. Identificação

In [104]:
print("Table: Phones_accelerometer - ID")
phone_acc.show(5)

Table: Phones_accelerometer - ID
+-----+-------------+-------------+----------+----------+--------+----+------+--------+-----+
|Index| Arrival_Time|Creation_Time|         x|         y|       z|User| Model|  Device|   gt|
+-----+-------------+-------------+----------+----------+--------+----+------+--------+-----+
|    0|1424696633908|1424696633908| -5.958191| 0.6880646|8.135345|   a|nexus4|nexus4_1|stand|
|    1|1424696633909|1424696633909|  -5.95224| 0.6702118|8.136536|   a|nexus4|nexus4_1|stand|
|    2|1424696633918|1424696633918|-5.9950867| 0.6535492|8.204376|   a|nexus4|nexus4_1|stand|
|    3|1424696633919|1424696633919|-5.9427185| 0.6761627|8.128204|   a|nexus4|nexus4_1|stand|
|    4|1424696633929|1424696633929| -5.991516|0.64164734|8.135345|   a|nexus4|nexus4_1|stand|
+-----+-------------+-------------+----------+----------+--------+----+------+--------+-----+
only showing top 5 rows



In [105]:
print("Table: Phones_gyroscope - ID")
phone_gyr.show(5)

Table: Phones_gyroscope - ID
+-----+-------------+-------------+-----------+-------------+------------+----+------+--------+-----+
|Index| Arrival_Time|Creation_Time|          x|            y|           z|User| Model|  Device|   gt|
+-----+-------------+-------------+-----------+-------------+------------+----+------+--------+-----+
|    0|1424696633909|1424696633909|0.013748169|-6.2561035E-4|-0.023376465|   a|nexus4|nexus4_1|stand|
|    1|1424696633909|1424696633909|0.014816284|-0.0016937256| -0.02230835|   a|nexus4|nexus4_1|stand|
|    2|1424696633918|1424696633918|  0.0158844|-0.0016937256|-0.021240234|   a|nexus4|nexus4_1|stand|
|    3|1424696633919|1424696633919|0.016952515| -0.003829956| -0.02017212|   a|nexus4|nexus4_1|stand|
|    4|1424696633928|1424696633928|  0.0158844|-0.0070343018| -0.02017212|   a|nexus4|nexus4_1|stand|
+-----+-------------+-------------+-----------+-------------+------------+----+------+--------+-----+
only showing top 5 rows



In [106]:
print("Table: Watch_accelerometer - ID")
watch_acc.show(5)

Table: Watch_accelerometer - ID
+-----+-------------+-------------+-----------+----------+-----------+----+-----+------+-----+
|Index| Arrival_Time|Creation_Time|          x|         y|          z|User|Model|Device|   gt|
+-----+-------------+-------------+-----------+----------+-----------+----+-----+------+-----+
|    0|1424696638740|1424696638740| -0.5650316| -9.572019|-0.61411273|   a| gear|gear_1|stand|
|    1|1424696638740|1424696638740|-0.83258367| -9.713276|-0.60693014|   a| gear|gear_1|stand|
|    2|1424696638740|1424696638740| -1.0181342| -9.935339|-0.54408234|   a| gear|gear_1|stand|
|    3|1424696638741|1424696638741| -1.2228385|-10.142437| -0.5662287|   a| gear|gear_1|stand|
|    4|1424696638741|1424696638741| -1.5771804|-10.480618|-0.40282443|   a| gear|gear_1|stand|
+-----+-------------+-------------+-----------+----------+-----------+----+-----+------+-----+
only showing top 5 rows



In [107]:
print("Table: Watch_gyroscope - ID")
watch_gyr.show(5)

Table: Watch_gyroscope - ID
+-----+-------------+-------------+-----------+------------+------------+----+-----+------+-----+
|Index| Arrival_Time|Creation_Time|          x|           y|           z|User|Model|Device|   gt|
+-----+-------------+-------------+-----------+------------+------------+----+-----+------+-----+
|    0|1424696638743|1424696638743|-0.16218652|-0.022104237|  0.05965481|   a| gear|gear_1|stand|
|    1|1424696638743|1424696638743|-0.18322548| -0.06178534| 0.012516857|   a| gear|gear_1|stand|
|    2|1424696638743|1424696638743|-0.18082865| -0.10865697|-0.036485307|   a| gear|gear_1|stand|
|    3|1424696638743|1424696638743|-0.14780544| -0.15792546| -0.09853696|   a| gear|gear_1|stand|
|    7|1424696638744|1424696638744| 0.18216023| -0.32357407| -0.27723506|   a| gear|gear_1|stand|
+-----+-------------+-------------+-----------+------------+------------+----+-----+------+-----+
only showing top 5 rows



Informações Sumarizadas:

- Quantos Registros cada Usuário Possui;
- Quais Modelos de Aparelho Cada Usuário Operou;
- Quais Devices cada usuário utilizou;
- Describe de x, y, z;
- Modelo de phone/watch mais utilizado;
- Devices mais utilizados;
- Gt mais utilizados;

In [108]:
print("Table: Phones_accelerometer - Stats")
aux1 = phone_acc.groupBy('User').agg(count('Index').alias('QT_RG_USU'),
                              mean('x').alias('VL_MED_X'),
                              mean('y').alias('VL_MED_Y'),
                              mean('z').alias('VL_MED_Z'))

aux2 = phone_acc.groupBy('User').pivot('Model').count()

aux3 = phone_acc.groupBy('User').pivot('Device').count()

aux4 = phone_acc.groupBy('User').pivot('gt').count()

phone_acc_sumarizadas = aux1.join(aux2,['User'],'inner')\
                            .join(aux3,['User'],'inner')\
                            .join(aux4,['User'],'inner')

phone_acc_sumarizadas.show()

Table: Phones_accelerometer - Stats
+----+---------+--------------------+--------------------+-----------------+------+------+------+----------+--------+--------+------+------+--------+--------+------------+------------+------+------+------+----------+--------+------+------+
|User|QT_RG_USU|            VL_MED_X|            VL_MED_Y|         VL_MED_Z|nexus4|    s3|s3mini|samsungold|nexus4_1|nexus4_2|  s3_1|  s3_2|s3mini_1|s3mini_2|samsungold_1|samsungold_2|  bike|  null|   sit|stairsdown|stairsup| stand|  walk|
+----+---------+--------------------+--------------------+-----------------+------+------+------+----------+--------+--------+------+------+--------+--------+------------+------------+------+------+------+----------+--------+------+------+
|   g|  1587697| -1.5738913443660654|  1.0314356575851165|9.159926074640264|733383|383313|199648|    271353|  361234|  372149|194851|188462|  186599|   13049|      136080|      135273|189810|309104|231827|    185210|  176814|224732|270200|
|   

In [109]:
print("Table: Phones_gyroscope - Stats")
aux1 = phone_gyr.groupBy('User').agg(count('Index').alias('QT_RG_USU'),
                              mean('x').alias('VL_MED_X'),
                              mean('y').alias('VL_MED_Y'),
                              mean('z').alias('VL_MED_Z'))

aux2 = phone_gyr.groupBy('User').pivot('Model').count()

aux3 = phone_gyr.groupBy('User').pivot('Device').count()

aux4 = phone_gyr.groupBy('User').pivot('gt').count()

phone_gyr_sumarizadas = aux1.join(aux2,['User'],'inner')\
                            .join(aux3,['User'],'inner')\
                            .join(aux4,['User'],'inner')

phone_gyr_sumarizadas.show()

Table: Phones_gyroscope - Stats
+----+---------+--------------------+--------------------+--------------------+------+------+------+--------+--------+------+------+--------+--------+------+------+------+----------+--------+------+------+
|User|QT_RG_USU|            VL_MED_X|            VL_MED_Y|            VL_MED_Z|nexus4|    s3|s3mini|nexus4_1|nexus4_2|  s3_1|  s3_2|s3mini_1|s3mini_2|  bike|  null|   sit|stairsdown|stairsup| stand|  walk|
+----+---------+--------------------+--------------------+--------------------+------+------+------+--------+--------+------+------+--------+--------+------+------+------+----------+--------+------+------+
|   g|  1718884|-0.00420692743481...|-0.00809072572438...|-0.00872708610152...|733389|785834|199661|  361226|  372163|400114|385720|  186615|   13046|195960|331523|259613|    197799|  190492|249330|294167|
|   f|  1309518|0.006516834609284814|-0.00733628632970...|-0.00689442369810...|736444|368315|204759|  374309|  362135|     1|368314|  179424|   

In [110]:
print("Table: Watch_accelerometer - Stats")
aux1 = watch_acc.groupBy('User').agg(count('Index').alias('QT_RG_USU'),
                              mean('x').alias('VL_MED_X'),
                              mean('y').alias('VL_MED_Y'),
                              mean('z').alias('VL_MED_Z'))

aux2 = watch_acc.groupBy('User').pivot('Model').count()

aux3 = watch_acc.groupBy('User').pivot('Device').count()

aux4 = watch_acc.groupBy('User').pivot('gt').count()

watch_acc_sumarizadas = aux1.join(aux2,['User'],'inner')\
                            .join(aux3,['User'],'inner')\
                            .join(aux4,['User'],'inner')

watch_acc_sumarizadas.show()

Table: Watch_accelerometer - Stats
+----+---------+------------------+------------------+-------------------+------+-------+------+------+---------+---------+------+------+-----+----------+--------+-----+-----+
|User|QT_RG_USU|          VL_MED_X|          VL_MED_Y|           VL_MED_Z|  gear|lgwatch|gear_1|gear_2|lgwatch_1|lgwatch_2|  bike|  null|  sit|stairsdown|stairsup|stand| walk|
+----+---------+------------------+------------------+-------------------+------+-------+------+------+---------+---------+------+------+-----+----------+--------+-----+-----+
|   g|   161141|-6.049081245130748|1.4073073373208311| -1.363632901525094| 47959| 113182| 37746| 10213|        1|   113181| 60525| 42906| 5940|     18138|    7358| 7230|19044|
|   f|   578939|-4.041330920178437|-4.430289090460987| 2.8105465789005155| 60995| 517944| 44670| 16325|   445594|    72350| 99680| 95316|70316|     74580|   85101|65382|88564|
|   e|   620188| 4.395599339498207| 4.019525086350341| 2.4994609574191586| 55859| 564

In [111]:
print("Table: Watch_gyroscope - Stats")
aux1 = watch_gyr.groupBy('User').agg(count('Index').alias('QT_RG_USU'),
                              mean('x').alias('VL_MED_X'),
                              mean('y').alias('VL_MED_Y'),
                              mean('z').alias('VL_MED_Z'))

aux2 = watch_gyr.groupBy('User').pivot('Model').count()

aux3 = watch_gyr.groupBy('User').pivot('Device').count()

aux4 = watch_gyr.groupBy('User').pivot('gt').count()

watch_gyr_sumarizadas = aux1.join(aux2,['User'],'inner')\
                            .join(aux3,['User'],'inner')\
                            .join(aux4,['User'],'inner')

watch_gyr_sumarizadas.show()

Table: Watch_gyroscope - Stats
+----+---------+--------------------+--------------------+--------------------+------+-------+------+------+---------+---------+-----+------+-----+----------+--------+-----+-----+
|User|QT_RG_USU|            VL_MED_X|            VL_MED_Y|            VL_MED_Z|  gear|lgwatch|gear_1|gear_2|lgwatch_1|lgwatch_2| bike|  null|  sit|stairsdown|stairsup|stand| walk|
+----+---------+--------------------+--------------------+--------------------+------+-------+------+------+---------+---------+-----+------+-----+----------+--------+-----+-----+
|   g|    92428|0.048284555968910965|-0.04307860292132773|-0.00844806716859...| 47711|  44717| 37564| 10147|     null|    44717|23222| 25882| 5920|     10755|    6165| 7122|13362|
|   f|   535078| 0.01269631705704994|-0.00604486080183...|-0.00908158557445256| 60808| 474270| 44559| 16249|   445594|    28676|89806| 85483|66391|     71626|   84222|65400|72150|
|   e|   567252|-0.00191504734200...|-0.00280758459968042|-0.01625163

Organizar os Campos de Arrival_Time e Creation_Time

In [112]:
print("Table: Phones_accelerometer")
phone_acc = phone_acc.withColumn('Arrival_Time_Stamp',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss"))
phone_acc = phone_acc.withColumn('Arrival_Time_MS',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))

phone_acc = phone_acc.withColumn('Creation_Time_Stamp',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss"))
phone_acc = phone_acc.withColumn('Creation_Time_NS',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))
phone_acc.show(5)

Table: Phones_accelerometer
+-----+-------------+-------------+----------+----------+--------+----+------+--------+-----+------------------+---------------+-------------------+----------------+
|Index| Arrival_Time|Creation_Time|         x|         y|       z|User| Model|  Device|   gt|Arrival_Time_Stamp|Arrival_Time_MS|Creation_Time_Stamp|Creation_Time_NS|
+-----+-------------+-------------+----------+----------+--------+----+------+--------+-----+------------------+---------------+-------------------+----------------+
|    0|1424696633908|1424696633908| -5.958191| 0.6880646|8.135345|   a|nexus4|nexus4_1|stand| 16-11-03 08:58:28|      000000000|  16-11-03 08:58:28|       000000000|
|    1|1424696633909|1424696633909|  -5.95224| 0.6702118|8.136536|   a|nexus4|nexus4_1|stand| 16-11-03 08:58:29|      000000000|  16-11-03 08:58:29|       000000000|
|    2|1424696633918|1424696633918|-5.9950867| 0.6535492|8.204376|   a|nexus4|nexus4_1|stand| 16-11-03 08:58:38|      000000000|  16-11-03 08:

In [113]:
print("Table: Phones_gyroscope")
phone_gyr = phone_gyr.withColumn('Arrival_Time_Stamp',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss"))
phone_gyr = phone_gyr.withColumn('Arrival_Time_MS',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))

phone_gyr = phone_gyr.withColumn('Creation_Time_Stamp',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss"))
phone_gyr = phone_gyr.withColumn('Creation_Time_NS',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))
phone_gyr.show(5)

Table: Phones_gyroscope
+-----+-------------+-------------+-----------+-------------+------------+----+------+--------+-----+------------------+---------------+-------------------+----------------+
|Index| Arrival_Time|Creation_Time|          x|            y|           z|User| Model|  Device|   gt|Arrival_Time_Stamp|Arrival_Time_MS|Creation_Time_Stamp|Creation_Time_NS|
+-----+-------------+-------------+-----------+-------------+------------+----+------+--------+-----+------------------+---------------+-------------------+----------------+
|    0|1424696633909|1424696633909|0.013748169|-6.2561035E-4|-0.023376465|   a|nexus4|nexus4_1|stand| 16-11-03 08:58:29|      000000000|  16-11-03 08:58:29|       000000000|
|    1|1424696633909|1424696633909|0.014816284|-0.0016937256| -0.02230835|   a|nexus4|nexus4_1|stand| 16-11-03 08:58:29|      000000000|  16-11-03 08:58:29|       000000000|
|    2|1424696633918|1424696633918|  0.0158844|-0.0016937256|-0.021240234|   a|nexus4|nexus4_1|stand| 16-1

In [114]:
print("Table: Watch_accelerometer")
watch_acc = watch_acc.withColumn('Arrival_Time_Stamp',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss"))
watch_acc = watch_acc.withColumn('Arrival_Time_MS',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))

watch_acc = watch_acc.withColumn('Creation_Time_Stamp',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss"))
watch_acc = watch_acc.withColumn('Creation_Time_NS',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))
watch_acc.show(5)

Table: Watch_accelerometer
+-----+-------------+-------------+-----------+----------+-----------+----+-----+------+-----+------------------+---------------+-------------------+----------------+
|Index| Arrival_Time|Creation_Time|          x|         y|          z|User|Model|Device|   gt|Arrival_Time_Stamp|Arrival_Time_MS|Creation_Time_Stamp|Creation_Time_NS|
+-----+-------------+-------------+-----------+----------+-----------+----+-----+------+-----+------------------+---------------+-------------------+----------------+
|    0|1424696638740|1424696638740| -0.5650316| -9.572019|-0.61411273|   a| gear|gear_1|stand| 16-11-03 10:19:00|      000000000|  16-11-03 10:19:00|       000000000|
|    1|1424696638740|1424696638740|-0.83258367| -9.713276|-0.60693014|   a| gear|gear_1|stand| 16-11-03 10:19:00|      000000000|  16-11-03 10:19:00|       000000000|
|    2|1424696638740|1424696638740| -1.0181342| -9.935339|-0.54408234|   a| gear|gear_1|stand| 16-11-03 10:19:00|      000000000|  16-11-0

In [115]:
print("Table: Watch_gyroscope")
watch_gyr = watch_gyr.withColumn('Arrival_Time_Stamp',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss"))
watch_gyr = watch_gyr.withColumn('Arrival_Time_MS',from_unixtime(col("Arrival_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))

watch_gyr = watch_gyr.withColumn('Creation_Time_Stamp',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss"))
watch_gyr = watch_gyr.withColumn('Creation_Time_NS',from_unixtime(col("Creation_Time"),"yy-MM-dd HH:mm:ss.SSSSSSSSS").substr(-9,9).cast('string'))
watch_gyr.show(5)

Table: Watch_gyroscope
+-----+-------------+-------------+-----------+------------+------------+----+-----+------+-----+------------------+---------------+-------------------+----------------+
|Index| Arrival_Time|Creation_Time|          x|           y|           z|User|Model|Device|   gt|Arrival_Time_Stamp|Arrival_Time_MS|Creation_Time_Stamp|Creation_Time_NS|
+-----+-------------+-------------+-----------+------------+------------+----+-----+------+-----+------------------+---------------+-------------------+----------------+
|    0|1424696638743|1424696638743|-0.16218652|-0.022104237|  0.05965481|   a| gear|gear_1|stand| 16-11-03 10:19:03|      000000000|  16-11-03 10:19:03|       000000000|
|    1|1424696638743|1424696638743|-0.18322548| -0.06178534| 0.012516857|   a| gear|gear_1|stand| 16-11-03 10:19:03|      000000000|  16-11-03 10:19:03|       000000000|
|    2|1424696638743|1424696638743|-0.18082865| -0.10865697|-0.036485307|   a| gear|gear_1|stand| 16-11-03 10:19:03|      00000

Informação das Diferenças dos Registros

In [116]:
from pyspark.sql.window import Window

In [117]:
%%time
print("Table: Phones_accelerometer")
window = Window.partitionBy(["User",'Device']).orderBy("Index")

phone_acc = phone_acc.withColumn("Arrival_Time_Diff", col("Arrival_Time") - lag(col("Arrival_Time"), 1, 0).over(window))
phone_acc = phone_acc.withColumn("Creation_Time_Diff", col("Creation_Time") - lag(col("Creation_Time"), 1, 0).over(window))

Table: Phones_accelerometer
CPU times: user 9.66 ms, sys: 1.53 ms, total: 11.2 ms
Wall time: 87.4 ms


In [118]:
%%time
print("Table: Phones_gyroscope")
window = Window.partitionBy(["User",'Device']).orderBy("Index")

phone_gyr = phone_gyr.withColumn("Arrival_Time_Diff", col("Arrival_Time") - lag(col("Arrival_Time"), 1, 0).over(window))
phone_gyr = phone_gyr.withColumn("Creation_Time_Diff", col("Creation_Time") - lag(col("Creation_Time"), 1, 0).over(window))

Table: Phones_gyroscope
CPU times: user 16.2 ms, sys: 1.37 ms, total: 17.6 ms
Wall time: 108 ms


In [119]:
%%time
print("Table: Watch_accelerometer")
window = Window.partitionBy(["User",'Device']).orderBy("Index")

watch_acc = watch_acc.withColumn("Arrival_Time_Diff", col("Arrival_Time") - lag(col("Arrival_Time"), 1, 0).over(window))
watch_acc = watch_acc.withColumn("Creation_Time_Diff", col("Creation_Time") - lag(col("Creation_Time"), 1, 0).over(window))

Table: Watch_accelerometer
CPU times: user 10.7 ms, sys: 2.88 ms, total: 13.6 ms
Wall time: 81.1 ms


In [120]:
%%time
print("Table: Watch_gyroscope")
window = Window.partitionBy(["User",'Device']).orderBy("Index")

watch_gyr = watch_gyr.withColumn("Arrival_Time_Diff", col("Arrival_Time") - lag(col("Arrival_Time"), 1, 0).over(window))
watch_gyr = watch_gyr.withColumn("Creation_Time_Diff", col("Creation_Time") - lag(col("Creation_Time"), 1, 0).over(window))

Table: Watch_gyroscope
CPU times: user 11.1 ms, sys: 1.25 ms, total: 12.4 ms
Wall time: 58.5 ms
